# Load Variables

In [1]:
%run "scripts/load_daily_variables.py"

Loading Variables: 100%|██████████| 26/26 [01:35<00:00,  3.66s/it]


# High Beta with Recent Elevated Volatility and Activity

In [2]:
hbeta_vol_regime_5day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-5:] >= 1.5).any()
                    ]
hbeta_vol_regime_20day = [sym for sym in symbols if 
                    (sym in results_finvizsearch.loc[results_finvizsearch['Beta'] > 2]['Ticker'].to_list()) and 
                    (symbols[sym].ATRs_Traded.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (symbols[sym].RVol.replace(np.inf, np.nan).dropna().iloc[-20:] >= 1.5).any() and
                    (sym not in hbeta_vol_regime_5day)
                    ]

- Quant Rating of stocks with elevated volatility in the last 5 days

In [3]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_5day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
LASR    4.981424
W       4.958204
GRAL    4.948916
AMKR    4.916409
FLNC    4.851393
GLXY    4.784830
APP     4.744582
NVTS    4.631579
DAVE    4.626935
EBS     4.619195
BFLY    4.193593
UCTT    4.160167
RIOT    4.009749
PACB    3.817549
VIK     3.811978
AMRC    3.706128
NCLH    3.611421
ONDS    3.495259
IREN    3.488148
CIFR    3.478666
CTMX    3.456993
ASTS    3.453945
RKT     3.443786
RDDT    3.440061
HOOD    3.424484
AMPX    3.421774
TMDX    3.410938
DPRO    3.266678
UMAC    3.231798
KOPN    3.170843
ACHR    3.141043
SHOP    3.139011
SPIR    3.125127
AEHR    3.092956
U       3.074331
WRBY    3.051304
PTON    3.037758
TREE    3.012022
LIND    2.928039
RYAM    2.892482
NVAX    2.873518
PDYN    2.867762
AEVA    2.846089
RDW     2.844734
OCGN    2.837623
JOBY    2.811209
APPS    2.795293
PCT     2.793600
TBCH    2.780393
PONY    2.749238
Name: 2026-01-20, dtype: float64

- Quant Rating of stocks with elevated volatility in the last 20 days minus those in hbeta_vol_regime_5day

In [4]:
mask = daily_quant_rating_df.index.isin(hbeta_vol_regime_20day)
col = daily_quant_rating_df.columns[-2]
(
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
.head(50)
)

Symbol
CRDO    4.953560
HUT     4.919505
TPC     4.868421
CTRN    4.866873
BTSG    4.795666
GAP     4.739938
SMTC    4.738390
AS      4.625387
PSIX    4.452646
CCL     4.402507
CUK     4.313370
CDNA    4.129526
SFIX    3.998607
INGM    3.909471
DAN     3.884401
LSPD    3.820334
FDMT    3.533426
SKYT    3.499323
RKLB    3.496275
PRAX    3.495936
BE      3.492889
SLS     3.491534
REAL    3.490857
COMP    3.490291
APLD    3.487132
NVDA    3.486116
LMND    3.477650
SYRE    3.453268
SOFI    3.433627
LC      3.430240
JMIA    3.407890
ADPT    3.406874
NBIS    3.375381
CENX    3.369285
OS      3.361497
ADEA    3.355401
TXG     3.299526
NXT     3.288351
Z       3.266990
SONO    3.220284
INOD    3.206062
AFRM    3.202675
BKSY    3.185743
IMNM    3.182696
OMER    3.173214
AMSC    3.154250
CVNA    3.117000
SAIA    3.107856
NAGE    3.099390
AUR     3.064172
Name: 2026-01-20, dtype: float64

# YTD Performance vs. Quant Rating

In [5]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = daily_quant_rating_df[quant_col].reset_index().rename(columns={quant_col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)

# Interest List Long Performance vs. Quant Rating

- Quant Rating for symbol in interest_list_long

In [6]:
mask = daily_quant_rating_df.index.isin(interest_list_long)
col = daily_quant_rating_df.columns[-2]
quantdf_illong = (
daily_quant_rating_df.loc[mask, col]
.sort_values(ascending=False)
)
quantdf_illong.head(50)

Symbol
MU      4.998452
TTMI    4.989164
NGD     4.987616
CIEN    4.982972
LASR    4.981424
TSM     4.970588
NUTX    4.967492
GOLD    4.962848
VSAT    4.959752
W       4.958204
AMZN    4.956656
NEM     4.955108
CRDO    4.953560
IAG     4.952012
GRAL    4.948916
GM      4.944272
GFI     4.942724
UPB     4.939628
LQDA    4.938080
CDE     4.936533
COHR    4.934985
INDV    4.933437
WLDN    4.931889
CLS     4.930341
BVN     4.927245
CSTM    4.922601
GH      4.921053
HUT     4.919505
DRH     4.916667
AMKR    4.916409
COF     4.914861
AU      4.908669
KGC     4.907121
INCY    4.905573
AA      4.904025
NBR     4.894737
RERE    4.890093
HBM     4.886997
CGAU    4.883901
ATI     4.876161
TSEM    4.874613
AEM     4.871517
CIB     4.869969
KRYS    4.862229
FLNC    4.851393
PAAS    4.845201
KALU    4.842105
ENVA    4.839009
VIAV    4.832817
LCII    4.826625
Name: 2026-01-20, dtype: float64

- Quant Rating vs. YTD Performance for symbols in interest_list_long

In [7]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = quantdf_illong.reset_index().rename(columns={col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')
# df = df.loc[df.Ticker.isin(fu.sector_industry_member_search('Basic Materials', level='sector'))]
# Assign color based on logic:
# quant rating > 4.8 & Performance < 50 -> green
# quant rating < 3 & Performance > 50 -> red
# else -> gray
def assign_color(row):
    if (row['Quant Rating'] > 4.8) and (row['Performance (YearToDate)'] < 50):
        return 'green'
    elif (row['Quant Rating'] < 3) and (row['Performance (YearToDate)'] > 50):
        return 'red'
    elif (row['Quant Rating'] > 2.5) and (row['Quant Rating'] < 4) and (row['Performance (YearToDate)'] > 50):
        return 'yellow'
    else:
        return 'gray'

df['Color'] = df.apply(assign_color, axis=1)

px.scatter(
    df.assign(
        Color_Label=df['Color'].map({'gray': 'uncertain', 'green': 'undervalued', 'red': 'overvalued', 'yellow': 'worth watching'})
    ),
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    color='Color_Label',
    color_discrete_map={'uncertain': 'gray', 'undervalued':'green', 'overvalued':'red', 'worth_watching': 'yellow'},
    category_orders={'Color_Label': ['uncertain', 'undervalued', 'overvalued', 'worth_watching']},
    labels={'Color_Label': 'Category'},
    title='YTD Performance vs. Quant Rating'
)

# INTEREST LIST LONG FROM FUNDAMENTALS

- Sorted by Quant Rating

In [8]:
interest_list_fundamental = [
    sym for sym in symbols if 
    (symbols[sym].interest_factor is not None) and
    ('Fundamental' in symbols[sym].interest_factor)
]
col = daily_quant_rating_df.columns[-2]
quantdf_ifund = (
daily_quant_rating_df.loc[daily_quant_rating_df.index.isin(interest_list_fundamental), col]
.sort_values(ascending=False)
)
quantdf_ifund.head(50)

Symbol
MU      4.998452
TTMI    4.989164
NGD     4.987616
CIEN    4.982972
LASR    4.981424
TSM     4.970588
NUTX    4.967492
GOLD    4.962848
VSAT    4.959752
W       4.958204
AMZN    4.956656
NEM     4.955108
CRDO    4.953560
IAG     4.952012
GRAL    4.948916
GM      4.944272
GFI     4.942724
UPB     4.939628
LQDA    4.938080
CDE     4.936533
COHR    4.934985
INDV    4.933437
WLDN    4.931889
CLS     4.930341
BVN     4.927245
CSTM    4.922601
GH      4.921053
HUT     4.919505
DRH     4.916667
AMKR    4.916409
COF     4.914861
AU      4.908669
KGC     4.907121
INCY    4.905573
AA      4.904025
Name: 2026-01-20, dtype: float64

- Sorted by Relative Volume

In [9]:
sorted(
    {sym: symbols[sym].RVol.iloc[-1].item() for sym in interest_list_fundamental}.items(),
    key=lambda x: x[1],
    reverse=True
)

[('IAG', 2.1703391675538404),
 ('KGC', 2.007924484853007),
 ('TSM', 1.7413161485932611),
 ('GFI', 1.7296125036949568),
 ('W', 1.5774999870397166),
 ('NGD', 1.5691939548934162),
 ('LASR', 1.569170187126167),
 ('MU', 1.5585969987491375),
 ('AU', 1.540648648139794),
 ('COF', 1.5253016623681992),
 ('GRAL', 1.513654473621442),
 ('GM', 1.4900861028863042),
 ('CDE', 1.4673703852529096),
 ('BVN', 1.3529700642262705),
 ('NEM', 1.2665367446012739),
 ('UPB', 1.2466658772868855),
 ('AMZN', 1.2347526255097085),
 ('VSAT', 1.2297658225644494),
 ('TTMI', 1.2212180347652268),
 ('DRH', 1.181988574641266),
 ('CIEN', 1.1769963995799977),
 ('CRDO', 1.1568263085642265),
 ('COHR', 1.1512567928465607),
 ('NUTX', 1.1390731796344231),
 ('AMKR', 1.1357388043878855),
 ('CSTM', 1.0977081697594508),
 ('AA', 1.0657311032485763),
 ('INCY', 1.0440613630789803),
 ('GH', 1.0359790072809494),
 ('LQDA', 1.006950537755588),
 ('GOLD', 0.9127882463780532),
 ('HUT', 0.8529895373533126),
 ('CLS', 0.8416113077963933),
 ('WLDN',

- Quant Rating vs. YTD Performance for symbols with fundamental factor in interest_list_long

In [10]:
import plotly.express as px
quant_col = daily_quant_rating_df.columns[-2]
quant_rating = quantdf_ifund.reset_index().rename(columns={col:'Quant Rating'})
perf = results_finvizsearch[['Ticker', 'Performance (YearToDate)']]
df = quant_rating.merge(perf, left_on='Symbol', right_on='Ticker', how='inner')

px.scatter(
    df,
    x='Performance (YearToDate)',
    y='Quant Rating',
    hover_data=['Ticker'],
    title='YTD Performance vs. Quant Rating'
)